# 第5章: 係り受け解析

日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

KNP使うときのコマンド

`juman < ai.ja.txt | knp -simple -anaphora > ai.ja.txt.parsed`

In [1]:
import os
from typing import List, Dict

In [2]:
input_path = os.path.join(os.getcwd(), '../data/ai.ja.txt.parsed')

In [3]:
# 読み込み
def get_raw(path: str) -> str:
    with open(path, mode='r') as f:
        return f.read()

In [4]:
# 行ごとに読み込み
def get_lines(path: str) -> List[str]:
    with open(path, mode='r') as f:
        s = [s.strip() for s in f.readlines()]
    return s

In [5]:
# 全文
raw = get_raw(input_path)

In [6]:
# 文リスト
sentences = raw.split('EOS\n')

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [7]:
# 形態素を表すクラス
class Morph():
    def __init__(self, morph_dict: dict):
        self.surface = morph_dict['surface'] # 表層形
        self.base = morph_dict['base'] # 基本形
        self.pos = morph_dict['pos'] # 品詞
        self.pos1 = morph_dict['pos1'] # 品詞細分類1

Morph({'surface': 'a', 'base': 'b', 'pos': 'c', 'pos1': 'd'})

In [8]:
# 入力形態素から始まる行を渡すと、表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）の辞書を返す
def get_surface_base_pos_pos1(text: str) -> dict:
    morp = text.split(' ')
    surface = morp[0]
    base = morp[2]
    pos = morp[3]
    pos1 = morp[5]
    return {
        'surface': surface,
        'base': base,
        'pos': pos,
        'pos1': pos1
    }

tes = 'a b c d e f'
expect = {
    'surface': 'a',
    'base': 'c',
    'pos': 'd',
    'pos1': 'f'
}
get_surface_base_pos_pos1(tes) == expect

True

In [9]:
# 1文を渡すと、形態素リストを返す
def get_sentence_morphs(sentence: str) -> List:
    # 改行で分ける
    # 長さ0の文字列は除外(末尾の改行によって空白になる要素)
    lines = [l for l in sentence.split('\n') if len(l) != 0]
    ret_list = []
    # 行ごとに処理
    for l in lines:
        # #, *, +で始まる行は形態素でない情報なのでパス
        if l[:1] == '#' or l[:1] == '*' or l[:1] == '+':
            continue
        # その他の（入力形態素から始まる）場合
        else:
            # 形態素を辞書型で取得
            morph_dict = get_surface_base_pos_pos1(l)
            # インスタンス作ってリストに入れる
            ret_list.append(Morph(morph_dict))
    return ret_list

tes = sentences[2]
''.join([m.surface for m in get_sentence_morphs(tes)])

'人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。'

In [10]:
# 出力
output = [get_sentence_morphs(s) for s in sentences]

In [12]:
# # 表示
# [(o.surface, o.base, o.pos, o.pos1) for o in output[2]]

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [126]:
# 文節を表すクラス
class Chunk():
    def __init__(self, chunk_dict: dict):
        self.morphs = chunk_dict['morphs']
        self.dst = chunk_dict['dst']
        self.srcs = chunk_dict['srcs']
    
    # 指定した表層形, 品詞を持っているかチェック
    def has_surface_pos(self, surface: str, pos: str) -> bool:
        return bool((surface, pos) in [(m.surface, m.pos) for m in self.morphs])
    
    # 指定した品詞を持っているかチェック
    def has_pos(self, check_str: str) -> bool:
        return bool(check_str in [m.pos for m in self.morphs])
    
    # 指定した品詞細分類1を持っているかチェック
    def has_pos1(self, check_str: str) -> bool:
        return bool(check_str in [m.pos1 for m in self.morphs])
    
    # 形態素の表層形をつなげた文字列を返す（特殊は除く）
    def get_morphs_surface(self):
        return ''.join([m.surface for m in self.morphs if m.pos != '特殊'])
    
    # 指定した品詞に合致する、最左の基本形を返す
    def get_pos_base(self, check_str: str) -> str:
        for m in self.morphs:
            if m.pos == check_str:
                return m.base
    
    # 指定した品詞に合致する、表層形を返す
    def get_pos_surface(self, check_str: str) -> str:
        for m in self.morphs:
            if m.pos == check_str:
                return m.surface
    
    # 指定した品詞細分類1に合致する、表層形を返す
    def get_pos1_surface(self, check_str: str) -> str:
        for m in self.morphs:
            if m.pos1 == check_str:
                return m.surface


In [127]:
import re
# 文節を渡すと、係り先と形態素リストを返す
def get_chunk_morphs(sentence: str) -> List:
    chunk = {}
    lines = sentence.split('\n')
    # 係り先
    chunk['dst'] = int(re.match(r'-*[0-9]+', lines[0]).group())
    # 形態素リスト
    chunk['morphs'] = get_sentence_morphs('\n'.join(lines[1:]))
    return chunk

In [128]:
# 1文を渡すと、文節リストを返す
def get_sentence_chunks(sentence: str) -> List:
    chunks_raw = sentence.split('\n* ')[1:] # 先頭は、#から始まる解析文の情報なので除外
    chunks = {}
    for i, c in enumerate(chunks_raw):
        chunks[i] = get_chunk_morphs(c)
        chunks[i]['srcs'] = []
    for i, c in chunks.items():
        if c['dst'] != -1:
            chunks[c['dst']]['srcs'].append(i)
    return [Chunk(c) for c in chunks.values()]

In [129]:
# # 表示
# [(c.dst, [m.surface for m in c.morphs]) for c in get_sentence_chunks(sentences[2])]

In [130]:
# 出力
output = [get_sentence_chunks(s) for s in sentences]
output = [o for o in output if len(o) != 0] # 長さ0の要素は除外

## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [28]:
def show_dst_srcs(chunks):
    for c in chunks:
        print(c.get_morphs_surface(),
              '\t',
              chunks[c.dst].get_morphs_surface(),
        )

show_dst_srcs(output[1])

人工知能 	 エーアイとは
じんこうちのう 	 エーアイとは
AI 	 エーアイとは
エーアイとは 	 指す
計算（）と 	 いう
いう 	 概念と
概念と 	 道具を
コンピュータ（）と 	 いう
いう 	 道具を
道具を 	 用いて
用いて 	 研究する
知能を 	 研究する
研究する 	 計算機科学（）の
計算機科学（）の 	 一分野を
一分野を 	 指す
指す 	 語
語 	 される
言語の 	 問題解決などの
理解や 	 推論
推論 	 問題解決などの
問題解決などの 	 知的行動を
知的行動を 	 行わせる
人間に 	 代わって
代わって 	 行わせる
コンピューターに 	 行わせる
行わせる 	 技術または
技術または 	 研究分野とも
計算機 	 コンピュータに
コンピュータに 	 よる
よる 	 情報処理システムの
知的な 	 情報処理システムの
情報処理システムの 	 実現に
設計や 	 実現に
実現に 	 関する
関する 	 研究分野とも
研究分野とも 	 される
される 	 される


In [29]:
# # 解答
# for chunks in output:
#     show_dst_srcs(chunks)

## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [32]:
def show_noun_verb(chunks):
    for c in chunks:
        # 係り先
        c_dst = chunks[c.dst]
        # 係り元に名詞がなければパス
        if c.has_pos(check_str='名詞') is False:
            continue
        # 係り先に動詞がなければパス
        if c_dst.has_pos(check_str='動詞') is False:
            continue
        print(
            c.get_morphs_surface(), # 係り元
            '\t',
            c_dst.get_morphs_surface(),
        )

show_noun_verb(output[1])

エーアイとは 	 指す
計算（）と 	 いう
コンピュータ（）と 	 いう
道具を 	 用いて
知能を 	 研究する
一分野を 	 指す
語 	 される
知的行動を 	 行わせる
人間に 	 代わって
コンピューターに 	 行わせる
コンピュータに 	 よる
実現に 	 関する
研究分野とも 	 される


In [20]:
# # 解答
# for chunks in output:
#     show_noun_verb(chunks)

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [34]:
from graphviz import Digraph

In [35]:
# 有向グラフ
dg = Digraph(format='png')
# 日本語表示
dg.attr('node', shape='box', fontname='MS Gothic')
# 中身
dg.node('あ')
dg.node('2')
dg.node('3')
dg.edge('あ', '2')  # 1 -> 2
dg.edge('2', '3')  # 2 -> 3
dg.edge('3', 'あ')  # 3 -> 1
dg.format = 'png'
dg.render(view = True)

'Digraph.gv.png'

In [36]:
def show_digraph(chunks, file_name: str = 'temp'):
    # 有向グラフ
    dg = Digraph(format='png')
    # 日本語表示
    dg.attr('node', shape='box', fontname='MS Gothic')
    # 中身
    for i, c in enumerate(chunks):
        # 係り元
        node1 = str(i) + '_' + c.get_morphs_surface()
        # ノード
        dg.node(node1)
        # 係り先
        if c.dst != -1: # 最後の文節は係り先がないので除外
            c_dst = chunks[c.dst]
            node2 = str(c.dst) + '_' + c_dst.get_morphs_surface()
            dg.edge(node1, node2)
    dg.format = 'png'
    dg.render(file_name)

show_digraph(output[3])

In [37]:
# # 解答
# for i, chunks in enumerate(output):
#     print(i)
#     show_digraph(chunks, str(i))

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

* コーパス中で頻出する述語と格パターンの組み合わせ
* 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [50]:
from functools import reduce

In [99]:
def get_predicate_postpositional(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        # 述語候補に動詞がなければパス
        if c.has_pos(check_str='動詞') is False:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        predicate = c.get_pos_base(check_str='動詞')
        
        # 格
        # 係り元一覧
        chunks_src = [chunks[s] for s in c.srcs]
        # 係り元の助詞一覧を取得する
        postpositional = [c.get_pos_surface(check_str='助詞') for c in chunks_src
                          if c.has_pos(check_str='助詞') # 助詞がある場合のみ
                         ]
        # 一つもなければパス
        if len(postpositional) == 0:
            continue
        ret_list.append([
            predicate,
            ' '.join(sorted(postpositional)), # スペース区切りで辞書順に並べる
        ])
    return ret_list

get_predicate_postpositional(output[1])

[['いう', 'と'],
 ['いう', 'と'],
 ['用いる', 'を'],
 ['する', 'を'],
 ['指す', 'と を'],
 ['代わる', 'に'],
 ['行う', 'に を'],
 ['よる', 'に'],
 ['関する', 'に'],
 ['する', 'と']]

In [100]:
# 出力
with open('45.txt', 'w') as f:
    for chunks in output:
        pred_posts = get_predicate_postpositional(chunks)
        for pred, post  in pred_posts:
            print(pred + '\t' + post, file=f)

In [67]:
# コーパス中で頻出する述語と格パターンの組み合わせ
# TOP10表示
!cut -f 1,2 45.txt | sort | uniq -c | sort -nr | head -10

     29 する	を
     24 いう	と
     22 する	と
     19 よる	に
     19 する	が
     10 する	は を
     10 する	で を
      8 する	
      7 する	に
      5 する	に は を


In [68]:
# 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順）
!cut -f 1,2 45.txt | sort | uniq -c | sort -nr | grep -e [0-9]*\s"行う" -e [0-9]*\s"なる" -e [0-9]*\s"*与える"

      3 行う	を
      3 行う	に を
      2 行う	は を
      2 なる	が に
      2 なる	が と
      1 与える	が など に
      1 与える	が
      1 行う	も を
      1 行う	まで を
      1 行う	に は は は
      1 行う	に
      1 行う	から は
      1 なる	に
      1 なる	と に は も
      1 なる	と
      1 なる	で に
      1 なる	が で に は は
      1 なる	が が と
      1 なる	から で と に は まで
      1 なる	


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

In [103]:
def get_predicate_postpositional_term(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        # 述語候補に動詞がなければパス
        if c.has_pos(check_str='動詞') is False:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        predicate = c.get_pos_base(check_str='動詞')
        
        # 格、項
        # 係り元一覧
        chunks_src = [chunks[s] for s in c.srcs]
        # 格、項を入れるリスト
        postpositional = []
        # 全ての係り元を調べる
        for c_src in chunks_src:
            # 助詞がなければパス
            if c_src.has_pos(check_str='助詞') is False:
                continue
            # 格、項を保存
            postpositional.append([
                c_src.get_pos_surface(check_str='助詞'), # 助詞
                c_src.get_morphs_surface(), # 文節の単語列
            ])
        # 一つもなければパス
        if len(postpositional) == 0:
            continue
        # 格でソート(多次元リストを普通にソートすると1番目の要素をキーにしてソートされる)
        postpositional = sorted(postpositional)
        # リストに入れる
        ret_list.append([
            predicate, # 述語
            ' '.join(p[0] for p in postpositional), # 格
            ' '.join(p[1] for p in postpositional), # 項
        ])
    return ret_list

get_predicate_postpositional_term(output[1])

[['いう', 'と', '計算（）と'],
 ['いう', 'と', 'コンピュータ（）と'],
 ['用いる', 'を', '道具を'],
 ['する', 'を', '知能を'],
 ['指す', 'と を', 'エーアイとは 一分野を'],
 ['代わる', 'に', '人間に'],
 ['行う', 'に を', 'コンピューターに 知的行動を'],
 ['よる', 'に', 'コンピュータに'],
 ['関する', 'に', '実現に'],
 ['する', 'と', '研究分野とも']]

In [78]:
# 出力
with open('46.txt', 'w') as f:
    for chunks in output:
        pred_posts = get_predicate_postpositional_term(chunks)
        for pred, post, term  in pred_posts:
            print(pred + '\t' + post + '\t' + term, file=f)

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

* 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
* 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
* 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

In [122]:
def get_predicate_postpositional_term_2(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        morphs1 = c.morphs
        # 述語候補に助詞がなければパス
        if '動詞' not in [m.pos for m in morphs1]:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        for m in morphs1:
            if m.pos == '動詞':
                predicate = m.base
        
        # 格、項
        morphs2 = [chunks[s].morphs for s in c.srcs]
        # 格、項を入れるリスト
        postpositional = []
        # 全ての係り元を調べる
        for m2 in morphs2:
            # を（助詞）がなければパス
            if ('を', '助詞') not in [(m.surface, m.pos) for m in m2]:
                continue
            # サ変名詞がなければパス
            if 'サ変名詞' not in [m.pos1 for m in m2]:
                continue
            # 助詞を探す
            for m in m2:
                if m.pos == '助詞':
                    post = m.surface # 格
                    predicate = m.surface + predicate # 述語に'を'を入れる
            # サ変名詞を探す
            for m in m2:
                if m.pos1 == 'サ変名詞':
                    predicate = m.surface + predicate # 述語に'サ変名詞'を入れる
            # 格、項を保存
            postpositional.append([
                post, # 格
                ''.join([m.surface for m in m2 if m.pos != '特殊']) # 項
            ])
        # 該当する文節がなかったらパス
        if len(postpositional) == 0:
            continue
        # 格でソート(多次元リストを普通にソートすると1番目の要素をキーにしてソートされる)
        postpositional = sorted(postpositional)
        
        # リストに入れる
        ret_list.append([
            predicate, # 述語
            ' '.join(p[0] for p in postpositional), # 格
            ' '.join(p[1] for p in postpositional), # 項
        ])
    return ret_list

get_predicate_postpositional_term_2(output[1])

[['行動を行う', 'を', '知的行動を']]

In [133]:
def get_predicate_postpositional_term_2(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        # 述語候補に動詞がなければパス
        if c.has_pos(check_str='動詞') is False:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        predicate = c.get_pos_base(check_str='動詞')
        
        # 格、項
        # 係り元一覧
        chunks_src = [chunks[s] for s in c.srcs]
        # 格、項を入れるリスト
        postpositional = []
        # 全ての係り元を調べる
        for c_src in chunks_src:
            # を（助詞）がなければパス
            if c_src.has_surface_pos(surface='を', pos='助詞') is False:
                continue
            # サ変名詞がなければパス
            if c_src.has_pos1(check_str='サ変名詞') is False:
                continue
            # 助詞
            post = c_src.get_pos_surface(check_str='助詞')
            predicate = post + predicate # 述語に'を'を入れる
            # サ変名詞
            predicate = c_src.get_pos1_surface(check_str='サ変名詞') + predicate # 述語にサ変名詞を入れる
            # 格、項を保存
            postpositional.append([
                post, 
                c_src.get_morphs_surface(), # 文節の単語列
            ])
        # 一つもなければパス
        if len(postpositional) == 0:
            continue
        # 格でソート(多次元リストを普通にソートすると1番目の要素をキーにしてソートされる)
        postpositional = sorted(postpositional)
        # リストに入れる
        ret_list.append([
            predicate, # 述語
            ' '.join(p[0] for p in postpositional), # 格
            ' '.join(p[1] for p in postpositional), # 項
        ])        
    return ret_list

get_predicate_postpositional_term_2(output[1])

[['行動を行う', 'を', '知的行動を']]

In [93]:
# 出力
with open('47.txt', 'w') as f:
    for chunks in output:
        pred_posts = get_predicate_postpositional_term_2(chunks)
        for pred, post, term  in pred_posts:
            print(pred + '\t' + post + '\t' + term, file=f)

## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

* 各文節は（表層形の）形態素列で表現する
* パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

In [138]:
def get_path_noun_2_root(chunks):
    ret_list = []
    for c in chunks:
        temp = []
        morphs = c.morphs
        # 名詞がなければパス
        if c.has_pos(check_str='名詞') is False:
            continue
        # 文節の単語列
        temp.append(c.get_morphs_surface())
        # 係り先
        num = c.dst
        while True:
            if num == -1:
                break
            morphs = chunks[num].morphs
            # 文節の単語列
            temp.append(chunks[num].get_morphs_surface())
            # 係り先を更新
            num = chunks[num].dst
        ret_list.append(' -> '.join(temp))
    return ret_list

get_path_noun_2_root(output[1])

['人工知能 -> エーアイとは -> 指す -> 語 -> される',
 'じんこうちのう -> エーアイとは -> 指す -> 語 -> される',
 'AI -> エーアイとは -> 指す -> 語 -> される',
 'エーアイとは -> 指す -> 語 -> される',
 '計算（）と -> いう -> 概念と -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '概念と -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 'コンピュータ（）と -> いう -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '知能を -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '研究する -> 計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '計算機科学（）の -> 一分野を -> 指す -> 語 -> される',
 '一分野を -> 指す -> 語 -> される',
 '語 -> される',
 '言語の -> 問題解決などの -> 知的行動を -> 行わせる -> 技術または -> 研究分野とも -> される',
 '理解や -> 推論 -> 問題解決などの -> 知的行動を -> 行わせる -> 技術または -> 研究分野とも -> される',
 '推論 -> 問題解決などの -> 知的行動を -> 行わせる -> 技術または -> 研究分野とも -> される',
 '問題解決などの -> 知的行動を -> 行わせる -> 技術または -> 研究分野とも -> される',
 '知的行動を -> 行わせる -> 技術または -> 研究分野とも -> される',
 '人間に -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される',
 'コンピューターに -> 行わせる -> 技術または -> 研究分野とも -> される',


## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

* 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
* 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

* 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
* 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示